# Task 1
## Modelos LeNet-5 y AlexNet


### 1. Implemente la arquitectura de LeNet-5 para resolver el problema de clasificación del daset de dígitos escritos a mano llamado mnist dataset

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
 
# Load MNIST data
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0,), (128,)),
])
train = torchvision.datasets.MNIST('data', train=True, download=True, transform=transform)
test = torchvision.datasets.MNIST('data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(train, shuffle=True, batch_size=100)
testloader = torch.utils.data.DataLoader(test, shuffle=True, batch_size=100)

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1131)>



100%|██████████| 9912422/9912422 [00:04<00:00, 1991870.78it/s]


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1131)>



100%|██████████| 28881/28881 [00:00<00:00, 381293.16it/s]


Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1131)>



100%|██████████| 1648877/1648877 [00:00<00:00, 2509311.70it/s]


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1131)>



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw



In [2]:
class LeNet5(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2) #6 channels of features as outputs
        self.act1 = nn.Tanh()
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0)
        self.act2 = nn.Tanh()
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(16, 120, kernel_size=5, stride=1, padding=0)
        self.act3 = nn.Tanh()

        self.flat = nn.Flatten() ##into a 2D vector
        self.fc1 = nn.Linear(1*1*120, 84)
        self.act4 = nn.Tanh()
        self.fc2 = nn.Linear(84, 10)
        
    def forward(self, x):
        # input 1x28x28, output 6x28x28
        x = self.act1(self.conv1(x))
        # input 6x28x28, output 6x14x14
        x = self.pool1(x)
        # input 6x14x14, output 16x10x10
        x = self.act2(self.conv2(x))
        # input 16x10x10, output 16x5x5
        x = self.pool2(x)
        # input 16x5x5, output 120x1x1
        x = self.act3(self.conv3(x))
        # input 120x1x1, output 84
        x = self.act4(self.fc1(self.flat(x)))
        # input 84, output 10
        x = self.fc2(x)
        return x

In [3]:
# Inicializar la red
model = LeNet5()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()



In [4]:
#Training and validation of the model

n_epochs = 10
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in trainloader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Validation
    model.eval()
    acc = 0
    count = 0
    for X_batch, y_batch in testloader:
        y_pred = model(X_batch)
        acc += (torch.argmax(y_pred, 1) == y_batch).float().sum()
        count += len(y_batch)
    acc = acc / count
    print("Epoch %d: model accuracy %.2f%%" % (epoch, acc*100))

Epoch 0: model accuracy 89.54%
Epoch 1: model accuracy 93.01%
Epoch 2: model accuracy 94.59%
Epoch 3: model accuracy 95.82%
Epoch 4: model accuracy 96.18%
Epoch 5: model accuracy 96.82%
Epoch 6: model accuracy 97.12%
Epoch 7: model accuracy 97.68%
Epoch 8: model accuracy 97.68%
Epoch 9: model accuracy 97.97%


### 2. Implemente la arquitectura de AlexNet para resolver el problema de clasificación usando el dataset de imagenes llamado CIFAR10 dataset

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [6]:
# Preprocesamiento del dataset CIFAR-10
transform = transforms.Compose([
    transforms.Resize(224),  # AlexNet espera entradas de tamaño 224x224
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)) # Normalización CIFAR-10
])

In [7]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)


100%|██████████| 170498071/170498071 [00:27<00:00, 6200691.40it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [8]:
# Definición de AlexNet
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [9]:
# Inicializar la red y los componentes para entrenamiento
model = AlexNet()
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()


In [ ]:
# Entrenamiento
n_epochs = 10
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in trainloader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validación
    model.eval()
    acc = 0
    count = 0
    with torch.no_grad():
        for X_batch, y_batch in testloader:
            y_pred = model(X_batch)
            acc += (torch.argmax(y_pred, 1) == y_batch).float().sum()
            count += len(y_batch)
    acc = acc / count
    print("Epoch %d: model accuracy %.2f%%" % (epoch, acc*100))

a. Diferencia principal entre ambas arquitecturas:             

La diferencia principal entre LeNet-5 y AlexNet radica en la complejidad y el tamaño. LeNet-5 fue diseñada para imágenes pequeñas y en escala de grises (MNIST, 28x28), mientras que AlexNet está optimizada para imágenes a color de mayor tamaño (ImageNet, 224x224). AlexNet tiene muchas más capas convolucionales, mayor cantidad de filtros y usa operaciones como ReLU y Dropout que permiten un mejor ajuste a tareas más complejas, mientras que LeNet-5 es una red más simple, adecuada para problemas menos exigentes.

b. ¿Podría usarse LeNet-5 para un problema como el que resolvió usando AlexNet? ¿Y viceversa?:          

LeNet-5 no sería ideal para resolver un problema como el de AlexNet, ya que está diseñada para manejar imágenes pequeñas y simples. Al trabajar con imágenes más grandes y complejas, como en CIFAR-10, no tendría suficiente capacidad de extracción de características.
Por otro lado, AlexNet podría resolver problemas sencillos como MNIST, pero sería excesivo. Utilizaría muchos más recursos de los necesarios y aumentaría el tiempo de entrenamiento sin aportar una mejora significativa.

c. Aspecto interesante de cada arquitectura:

De LeNet-5 me parece interesante su simplicidad y eficiencia en problemas pequeños como la clasificación de dígitos. Es rápida y logra una precisión muy alta en MNIST con una estructura relativamente simple.
De AlexNet lo más interesante es cómo maneja imágenes complejas y a color. Me llama la atención cómo el uso de ReLU y Dropout ayudó a mejorar significativamente el rendimiento en problemas de clasificación de imágenes grandes y variadas como las de ImageNet o CIFAR-10.